[과제함](https://cafe.naver.com/esaa2019?iframe_url_utf8=%2FArticleRead.nhn%253Fclubid%3D29858915%2526page%3D1%2526menuid%3D90%2526boardtype%3DL%2526articleid%3D2708%2526referrerAllArticles%3Dfalse)

기한: 3월 22일 (금) 18:30\
​
과제: 핸즈온 7장 앙상블 학습과 랜덤 포레스트\
​
p. 245-271 필사하여 깃허브에 업로드한 뒤, 깃허브 링크를 댓글로 달아주세요.
​
필사 전 코드 추가)

In [1]:
#필사 전 코드 추가
import warnings
warnings.filterwarnings('ignore')
# import package
import numpy as np
import os
#5장에서 소개한 moons dataset 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

p.251\
수정 전)

```
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, n_jobs=-1
)
```


수정 후)


```
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1
)
```




앙상블: 일련의 예측기 (즉, 분류나 회귀 모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있음.

(예) 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정 트리 분류기를 훈련시킬 수 있습니다. 모든 개별 트리의 예측을 구하고가장 많은 선택을 받은 클래스를 예측으로 삼음


# 1) 투표 기반 분류기

* 직접 (hard) 투표 분류기 : 각 분류기의 예측을 모아서 가장 많이 선택된 클래스를 예측하는 것
* 간접 (soft) "" : 모든 분류기가 클래스의 확률을 예측할 수 있으면 (즉, predict_proba() 메서드가 있으면) 개별 분류기의 예측을 평균 내어 가장 높은 클래스를 예측


간접 방식이 확률이 높은 투표에 비중을 더 두기 때문에 직접보다 성능이 높음

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# import package
import numpy as np
import os

#5장에서 소개한 moons dataset 불러오기
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
X,y = make_moons(n_samples=100, noise=0.15)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf= SVC()

voting_clf = VotingClassifier(
    estimators = [("lr", log_clf), ("rf", rnd_clf), ("svc", svm_clf)],
    voting = "hard"
)

voting_clf.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [ ]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test,y_pred))

LogisticRegression 0.95
RandomForestClassifier 1.0
SVC 1.0
VotingClassifier 1.0


# 2)  배깅과 페이스팅


- 배깅 (bootstrap aggregating): 훈련세트에서 중복을 허용하여 샘플링하는 방식

- 페이스팅: 중복을 허용하지 않고 샘플링하는 방식


모든 예측기가 훈련을 마치면 앙상블은 모든 예측기의 예측을 모아서 새로운 샘플에 대한 예측을 만듭니다. 수집 함수는 전형적으로
* 분류일 때는 통계적 최빈값
* 회귀에 대해서는 평균을 계산합니다.

개별 예측기는 원본 훈련 세트로 훈련시킨 것보다 훨씬 크게 편향되어 있지만 수집 함수를 통과하면 편향과 분산이 모두 감소\
일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킨 것보다 더 좋음

## 2.1 사이킷런의 배깅과 페이스팅

* 배깅: boostrap = True
* 페이스팅: False

Note: BaggingClassifier는 기반이 되는 분류기가 결정 트리 분류기처럼 클래스 확률을 추정할 수 있으면 (즉, predict_proba() 함수가 있으면) 직접 투표 대신 자동으로 간접 투표 방식을 사용합니다.

* 배깅이 페이스팅보다 편향이 조금 더 높음\
(부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로)
* 하지만 다양성을 추가한다는 것은 예측기들의 상관관계를 줄이므로 앙상블의 분산을 감소시킴
* 전반적으로 배깅이 더 나은 모델을 만들기 때문에 일반적으로 더 선호
* 그러나 시간과 CPU 파워에 여유가 있다면 교차 검증으로 배깅과 페이스팅을 모두 평가해서 더 나은 쪽을 선택하는 것이 좋습니다.

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    max_samples=50, bootstrap=True, n_jobs=-1
)


bag_clf.fit(X_train, y_train)

y_pred = bag_clf.predict(X_test)

## 2.2 oob 평가

* 배깅을 사용하면 어떤 샘플은 한 예측기를 위해 여러 번 샘플링되고 어떤 것은 전혀 선택되지 않을 수 있음
* BaggingClassifier는 기본값으로 중복을 허용하여 (bootstrap=True) 훈련 세트의 크기만큼인 m개 샘플을 선택함\
(평균적으로 각 예측기에 훈련 샘플의 약 63% 정도만 샘플링된다는 뜻)
* 선택되지 않은 훈련 샘플의 나머지 37% = OOB (Out-of-Bag) 샘플\
(예측기마다 남겨진 OOB sample은모두 다름)

BaggingClassifier를 만들 때 oob_score=True로 지정하면 훈련이 끝난 후 자동으로 oob 평가를 수행

In [ ]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
     bootstrap=True, n_jobs=-1, oob_score=True
)

bag_clf.fit(X_train, y_train)


BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=500,
                  n_jobs=-1, oob_score=True)

In [ ]:
bag_clf.oob_score_

0.9

In [ ]:
from sklearn.metrics import accuracy_score

y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

1.0

In [ ]:
bag_clf.oob_decision_function_

array([[0.67708333, 0.32291667],
       [0.        , 1.        ],
       [0.76470588, 0.23529412],
       [0.97979798, 0.02020202],
       [1.        , 0.        ],
       [0.96216216, 0.03783784],
       [0.        , 1.        ],
       [0.84916201, 0.15083799],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.94210526, 0.05789474],
       [0.        , 1.        ],
       [1.        , 0.        ],
       [0.69230769, 0.30769231],
       [0.32402235, 0.67597765],
       [0.00552486, 0.99447514],
       [0.        , 1.        ],
       [0.16860465, 0.83139535],
       [0.21465969, 0.78534031],
       [0.02185792, 0.97814208],
       [0.97515528, 0.02484472],
       [0.50802139, 0.49197861],
       [0.19895288, 0.80104712],
       [0.99494949, 0.00505051],
       [0.00543478, 0.99456522],
       [0.96022727, 0.03977273],
       [0.67525773, 0.32474227],
       [0.02259887, 0.97740113],
       [0.22807018, 0.77192982],
       [0.

# 3) 랜덤 패치와 랜덤 서브스페이스

BaggingClassifier는 특성 샘플링도 지원함\
* 특성 샘플링은 더 다양한 예측기를 만들며 편향을 늘리는 대신 분산을 낮춤

샘플링은 두 매개변수로 조절됨
* `max_features`
* `bootstrap_features`

작동 방식은 `max_samples`, `bootstrap`과 동일하지만 샘플이 아니고 **특성에 대한 샘플링**이다

따라서 각 예측기는 무작위로 선택한 입력 특성의 일부분으로 훈련됩니다.

이 기법은 특히 (이미지와 같은) 매우 고차원의 데이터셋을 다룰 때 유용
* 랜덤 패치 방식 (random patches method): 훈련 샘플과 샘플을 모두 샘플링하는 것
* 랜덤 서브스페이스 방식 (random subspaces method): 훈련 샘플을 모두 사용하고 (bootstrap=False이고 max_samples=1.0로 설정) 특성은 샘플링하는 (bootstrap_features=True 그리고/또는 max_features는 1.0보다 작게 설정) 것



# 4) 랜덤 포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)
rnd_clf.fit(X_train,y_train)

y_pred_rf = rnd_clf.predict(X_test)

In [ ]:
# BaggingClassifier를 사용해 앞의 RandomForestClassifier와 거의 유사하게 만든 것

bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="auto", max_leaf_nodes=16),
    n_estimators=500, max_samples=1.0, bootstrap=True, n_jobs=-1
)

## 4.1 Extra-trees (Extremely Randomized tree ensemble)

사이킷런의 ExtraTreesClassifier, 사용법은 RandomForestClasifier 과 동일

* 무작위한 트리의 랜덤 포레스트
* 여기서도 역시 편향이 늘어나지만 대신 분산을 낮추게 됨
* 일반적인 랜덤 포레스트보다 엑스트라 트리가 훨씬 빠름\
(모든 노드에서 특성마다 가장 최적의 임곗값을 찾는 것이 트리 알고리즘에서 가장 시간이 많이 소요되는 작업 중 하나이기 때문)

In [ ]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier

## 4.2 feature_importances

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()
rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)
rnd_clf.fit(iris["data"], iris["target"])

for name,score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.09739409711646302
sepal width (cm) 0.022851475580623188
petal length (cm) 0.41726283274356457
petal width (cm) 0.46249159455934924


# 5) 부스팅

약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법

## 5.1. 에이다부스트

이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높인다.\
그러면 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞춰지게 됨

Caution:
연속된 학습 기법의 중요한 단점은 병렬화 또는 분할을 할 수 없는 것이다.\
이전 예측기가 훈련되고 평가된 후에 각 예측기가 학습될 수 있기 때문이다.\
그래서 배깅이나 페이스팅보다 확장성이 떨어짐.

원래 에이다부스트 알고리즘은 학습률 하이퍼파라미터를 사용하지 않음

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5

)

ada_clf.fit(X_train,y_train)

AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200)

## 5.2. 그레이디언트 부스팅

에이다부스트처럼 그레이디언트 부스팅은 앙상블에 이전까지의 오차를 보정하도록 예측기를 순차적으로 추가.\
하지만 에이다부스트처럼 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 잔여 오차에 새로운 예측기를 학습시킴

In [ ]:
import numpy as np

# 잡음이 섞인 2차 곡선 형태의 데이터 생성
np.random.seed(42)  # 재현성을 위해 난수 시드 설정

# 특성 데이터 생성 (0에서 1까지 균일하게 분포된 100개의 데이터)
X = 2 * np.random.rand(100, 1)

# 타깃 데이터 생성 (y = 0.5 * X^2 + X + 잡음)
y = 0.5 * X**2 + X + np.random.randn(100, 1)

# X와 y 출력
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (100, 1)
y shape: (100, 1)


In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2)
tree_reg1.fit(X,y)

DecisionTreeRegressor(max_depth=2)

In [ ]:
y2 = y-tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [ ]:
y3 = y-tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y3)

DecisionTreeRegressor(max_depth=2)

In [ ]:
X_new = np.linspace(0, 1, 100).reshape(-1, 1)

In [ ]:
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X,y)

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120)
gbrt.fit(X_train,y_train)

errors = [mean_squared_error(y_val,y_pred)
         for y_pred in gbrt.staged_predict(X_val)]

bst_n_estimators=np.argmin(errors)+1

gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators)
gbrt_best.fit(X_train,y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=19)

In [ ]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf")
error_going_up = 0

for n_estimators in range(1,120):
    gbrt.n_estimators=n_estimators
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)
    if val_error < min_val_error:
        min_val_error = val_error
        error_going_up=0
    else:
        error_going_up+=1
        if error_going_up==5:
            break

In [ ]:
import xgboost

xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

In [ ]:
xgb_reg.fit(X_train, y_train,
           eval_set=[(X_val,y_val)], early_stopping_rounds=2)

y_pred = xgb_reg.predict(X_val)

[0]	validation_0-rmse:1.53028
[1]	validation_0-rmse:1.38567
[2]	validation_0-rmse:1.32343
[3]	validation_0-rmse:1.26008
[4]	validation_0-rmse:1.23646
[5]	validation_0-rmse:1.23243
[6]	validation_0-rmse:1.21633
[7]	validation_0-rmse:1.21944
[8]	validation_0-rmse:1.21959


# 6) 스태킹